In [1]:
from os import path

from matplotlib import pyplot as plt
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, make_union
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils.class_weight import compute_sample_weight
from sklearn import metrics

from functions import load_bad_words, build_data_path, print_report
from constants import LABEL_COLS

In [2]:
BAD_WORDS = load_bad_words()
training_data_path = build_data_path('train.csv')

In [3]:
df = pd.read_csv(training_data_path)
X = df['comment_text']
y = df[LABEL_COLS]

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4)

In [5]:
clf = RandomForestClassifier()

tfidf = TfidfVectorizer(lowercase=True, stop_words='english')
bad_word_counter = CountVectorizer(vocabulary=BAD_WORDS)

union = make_union(tfidf, bad_word_counter)

pipeline = make_pipeline(union, clf)

optimizer = pipeline

print('Would you like to perform an exhaustive search? NOTE: This will take several hours.')
autotune_hyperparameters = input('Please enter "yes" or "no".')


# Auto-tune hyperparameters
while autotune_hyperparameters.lower() not in ['yes', 'no']:
    autotune_hyperparameters = input('Please enter "yes" or "no".')
if autotune_hyperparameters == 'yes':
    parameters = {
        'featureunion__tfidfvectorizer__lowercase': [True, False],
        'featureunion__tfidfvectorizer__max_features': [1000, 5000, 10000, None],
        'featureunion__countvectorizer__binary': [True, False],
        'randomforestclassifier__class_weight': [None, 'balanced'],
    }
    optimizer = GridSearchCV(pipeline, parameters, scoring='f1_weighted', verbose=3)
fit_params = {
    'randomforestclassifier__sample_weights': compute_sample_weight('balanced', y_train)
}
optimizer.fit(X_train, y_train)

Would you like to perform an exhaustive search? NOTE: This will take several hours.
Please enter "yes" or "no".yes


/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] featureunion__countvectorizer__binary=True, featureunion__tfidfvectorizer__lowercase=True, featureunion__tfidfvectorizer__max_features=1000, randomforestclassifier__class_weight=None 


/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[CV]  featureunion__countvectorizer__binary=True, featureunion__tfidfvectorizer__lowercase=True, featureunion__tfidfvectorizer__max_features=1000, randomforestclassifier__class_weight=None, score=0.6460078991624626, total=  46.1s
[CV] featureunion__countvectorizer__binary=True, featureunion__tfidfvectorizer__lowercase=True, featureunion__tfidfvectorizer__max_features=1000, randomforestclassifier__class_weight=None 


...

[CV]  featureunion__countvectorizer__binary=False, featureunion__tfidfvectorizer__lowercase=False, featureunion__tfidfvectorizer__max_features=None, randomforestclassifier__class_weight=balanced, score=0.4624154642399889, total=  54.3s
[CV] featureunion__countvectorizer__binary=False, featureunion__tfidfvectorizer__lowercase=False, featureunion__tfidfvectorizer__max_features=None, randomforestclassifier__class_weight=balanced 


/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[CV]  featureunion__countvectorizer__binary=False, featureunion__tfidfvectorizer__lowercase=False, featureunion__tfidfvectorizer__max_features=None, randomforestclassifier__class_weight=balanced, score=0.4467570793450672, total= 1.1min


[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed: 75.1min finished
/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('featureunion', FeatureUnion(n_jobs=None,
       transformer_list=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'featureunion__tfidfvectorizer__lowercase': [True, False], 'featureunion__tfidfvectorizer__max_features': [1000, 5000, 10000, None], 'featureunion__countvectorizer__binary': [True, False], 'randomforestclassifier__class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=3)

In [6]:
y_predictions = optimizer.predict(X_valid)
print(y_predictions.shape, y_valid.shape)
# best_estimator_ = optimizer.best_estimator_

(63829, 6) (63829, 6)


In [7]:
metrics.roc_auc_score(y_valid, y_predictions)

0.6874419647275994

In [8]:
test_data = build_data_path('test.csv')

data_df = pd.read_csv(test_data)

test_labels = build_data_path('test_labels.csv')
label_df = pd.read_csv(test_labels)

test_df = data_df.set_index('id').join(label_df.set_index('id'))
CONDITIONS = [f'{label} != -1' for label in LABEL_COLS]
QUERY_STRING = ' & '.join(CONDITIONS)
test_df = test_df.query(QUERY_STRING)
X_test = test_df['comment_text']
y_test = test_df[LABEL_COLS]

y_predictions = optimizer.predict(X_test)

print_report(y_test, y_predictions, data_type='TESTING')

TESTING RESULTS:

               precision    recall  f1-score   support

        toxic       0.55      0.78      0.64      6090
 severe_toxic       0.17      0.10      0.12       367
      obscene       0.56      0.72      0.63      3691
       threat       0.29      0.13      0.18       211
       insult       0.52      0.57      0.55      3427
identity_hate       0.59      0.15      0.24       712

    micro avg       0.54      0.66      0.59     14498
    macro avg       0.45      0.41      0.39     14498
 weighted avg       0.53      0.66      0.58     14498
  samples avg       0.07      0.06      0.06     14498

Class-wise AUC-ROC (Kaggle) [0.85496713 0.54635554 0.84020997 0.56345569 0.77155785 0.57455562]
Overall AUC-ROC (Kaggle) 0.6918502981032894


/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
